<a href="https://colab.research.google.com/github/JosephSilva362/Webscarping-Transformacao/blob/main/Teste_joseph_cunha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar a biblioteca Tabula

In [ ]:
pip install tabula-py

In [ ]:
#Import das bibliotecas necessárias

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import tabula
import pandas as pd
from zipfile import ZipFile


## Teste 1 - WebScraping

In [ ]:
#Função para baixar o arquivo
def download_file(url, filename):
    file_link = requests.get(url, stream=True)
    if file_link.status_code == requests.codes.OK:
        with open(filename, 'wb') as nf:
            for chunk in file_link.iter_content(chunk_size=256):
                nf.write(chunk)
        print("Download finalizado")
    else:
        file_link.raise_for_status()

#Função para encontrar o arquivo
def find_file(url):
    #Guardar o HTML em um arquivo bs4
    soup = bs(requests.get(url).content, 'html.parser')
    #Encontrar o link para a página de download do arquivo mais recente
    download_page = soup.find('a', attrs={'class':'alert-link internal-link'})['href']
    #Guarda o HTML da página de download
    soup = bs(requests.get(download_page).content, 'html.parser')
    #Encontra o link para baixar o arquivo
    download_link = soup.find('table').find('a')['href']
    #O nome do arquivo pode ser encontrado na ultima parte da url
    filename = download_link.split('/')[-1]
    #Chamar a função de download
    download_file(download_link, filename)

#main
if __name__ == "__main__":
    URL = 'https://www.gov.br/ans/pt-br/assuntos/prestadores/padrao-para-troca-de-informacao-de-saude-suplementar-2013-tiss'
    find_file(URL)

Download finalizado


## Teste 2 - Transformação de dados

In [ ]:
#Ler o pdf

In [ ]:
pdf = 'padrao-tiss_componente-organizacional_202111.pdf'

###Construindo a tabela do quadro 30

In [ ]:
#Extrair o quadro 30

In [ ]:
quadro30 = tabula.read_pdf(pdf, pages='114')
tabela = quadro30[0]

In [ ]:
display(tabela)

In [ ]:
#Separar a coluna "Tabela de Tipo do Demandante" em duas: "Código" e "Descrição da categoria"

In [ ]:
tabela[["Código", "Descrição da categoria"]] = tabela["Tabela de Tipo do Demandante"].str.split(" ", 1, expand=True)


In [ ]:
#Excluir a coluna "Tabela de Tipo do Demandante"

In [ ]:
tabela.drop(['Tabela de Tipo do Demandante'],axis=1,inplace=True)

In [ ]:
#Excluir a linha excedente 

In [ ]:
tabela.drop(0,inplace=True)

In [ ]:
#Definir a coluna "Código" como o index

In [ ]:
tabela.set_index("Código",inplace=True)

In [ ]:
#Salva o dataframe como um arquivo csv

In [ ]:
tabela.to_csv('quadro30.csv',sep=",", encoding='utf-8-sig')

###Construindo a tabela do quadro 31

In [ ]:
#Extrair o quadro 31

In [ ]:
quadro31 = tabula.read_pdf(pdf, pages=[i for i in range(115,121)], pandas_options={'header':None})

In [ ]:
for i in quadro31:
  display(i)

In [ ]:
#Excluir a ultima tabela

In [ ]:
quadro31.pop(-1)

In [ ]:
#Criar um dataframe para receber o quadro 31 que está dividido

In [ ]:
tabela = pd.DataFrame([])

In [ ]:
#Juntar o quadro 31 em um dataframe

In [ ]:
for df in quadro31:
  tabela = pd.concat([tabela,df])

In [ ]:
#Excluir as linhas excedentes

In [ ]:
tabela.drop([0,1],inplace=True)

In [ ]:
#definir o cabeçalho

In [ ]:
tabela.columns = ['Código','Descrição da categoria']

In [ ]:
#definir a coluna "Código" como o Index

In [ ]:
tabela.set_index("Código",inplace=True)

In [ ]:
display(tabela)

In [ ]:
#Resolve o problema de quebra de linha

In [ ]:
tabela['Descrição da categoria'].replace('\r', ' ', regex=True, inplace=True)

In [ ]:
#Salva o arquivo como csv

In [ ]:
tabela.to_csv('quadro31.csv',sep=",", encoding='utf-8-sig')

###Construindo a tabela do Quadro 32

In [ ]:
#Extrair o quadro 32

In [ ]:
quadro32 = tabula.read_pdf(pdf, pages='120')
tabela = quadro32[1]

In [ ]:
display(tabela)

In [ ]:
#Separar a coluna "Tabela de Tipo de Solicitação" em duas colunas "Código" e "Descrição da categoria"

In [ ]:
tabela[["Código", "Descrição da categoria"]] = tabela["Tabela de Tipo de Solicitação"].str.split(" ", 1, expand=True)

In [ ]:
#Excluir a coluna e as linhas excedentes

In [ ]:
tabela.drop(['Tabela de Tipo de Solicitação'],axis=1,inplace=True)

In [ ]:
tabela.drop([0,1, 4], inplace=True)

In [ ]:
#Definir a coluna "Código" como o index

In [ ]:
tabela.set_index("Código",inplace=True)

In [ ]:
#Salva o arquivo como csv

In [ ]:
tabela.to_csv('quadro32.csv',sep=",", encoding='utf-8-sig')

###Comprimindo todos os CSVs em um arquivo .zip

In [ ]:
with ZipFile('Teste_Joseph_Silva_da_Cunha.zip','w') as my_zip:
    my_zip.write('quadro30.csv')
    my_zip.write('quadro31.csv')
    my_zip.write('quadro32.csv')
    my_zip.close()